# Implement a new Method

You can try this tutorial on [Google Colab](https://colab.research.google.com/github/TUMT2022/LDIMBench/blob/main/docs/user-guides/run-your-method.ipynb)!

This guide will show you how to develop your own leakage detection method (in python, there are [other options](/docs/user-guides/docker.md) as well!) and benchmark it against the other methods in the benchmark suite.

First install the package:

In [1]:
# Run this in Google Colab:
!python3 --version
!pip uninstall google-colab community flask -y  # Workaround for GoogleColab beeing out of date
!pip install ipython ipykernel --upgrade        # https://github.com/googlecolab/colabtools/issues/2230
!pip install ldimbenchmark

     |████████████████████████████████| 40 kB 2.5 MB/s eta 0:00:011
     |████████████████████████████████| 12.2 MB 6.4 MB/s eta 0:00:01
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached big_O-0.10.2.tar.gz (10 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
     |████████████████████████████████| 4.0 MB 74.9 MB/s eta 0:00:01
     |████████████████████████████████| 9.7 MB 92.3 MB/s eta 0:00:01
  Using cached docker-6.0.1-py3-none-any.whl (147 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
     |████████████████████████████████| 13.6 MB 96.6 MB/s eta 0:00:01
     |████████████████████████████████| 701 kB 108.9 MB/s eta 0:00:01
  Using cached pytz-2022.6-py2.py3-none-any.whl (498 kB)
     |████████████████████████████████| 17.1 MB 81.8 MB/s eta 0:00:01
     |████████████████████████████████| 33.8 MB 106.0 MB/s eta 0:00:01
  Using cached networkx-2.8.8-py3-none-any.whl (2.0 MB)
     |████████████████████████████████| 9.4 MB 107.0 MB/s eta 0:00:01  

Define your Leakage Detection Method (here you can read more about the [programming interface](/docs/design/programming-interface.md)):


In [1]:
from ldimbenchmark import LDIMMethodBase, BenchmarkData, BenchmarkLeakageResult
from ldimbenchmark.classes import MethodMetadata, Hyperparameter
from typing import List
import random

# Your method should inherit from LDIMMethodBase
# It should implement all abstract methods
class MyNewLeakageDetectionMethod(LDIMMethodBase):
    """
    MyNewLeakageDetectionMethod
    """

    def __init__(self):
        # Provide information about your method in the super call
        super().__init__(
            name="NEWMETHOD",
            version="1.0",
            metadata=MethodMetadata(
                data_needed=["pressures", "demands", "flows", "levels"],
                hyperparameters=[
                    Hyperparameter(
                        name="random",
                        description="The Random percentage of detecing a leakage",
                        default=0.5,
                        max=1.0,
                        min=0.0,
                        type=float,
                    ),
                ],
            )
        )

    def train(self, train_data: BenchmarkData) -> None:
        return

    def detect(self, evaluation_data: BenchmarkData) -> List[BenchmarkLeakageResult]:
        return []

    def detect_datapoint(self, evaluation_data) -> BenchmarkLeakageResult:
        return BenchmarkLeakageResult(
                    leak_pipe_id="<your_pipe_id>",
                    leak_pipe_nodes="<your_leak_pipe_nodes>",
                    leak_node="",
                    leak_diameter="",
                    leak_area="",
                    leak_type="",
                    leak_time_start="",
                    leak_time_peak="",
                    leak_time_end="",
                ) if random() < 0.5 else None


To test your method we need a dataset. We provide a few datasets in the [datasets namespace](#). You can also use your own dataset (see [run on your dataset guide](/docs/user-guides/run-on-your-dataset.ipynb)).

In [2]:
from ldimbenchmark.datasets import Dataset, DatasetLibrary, DATASETS

datasets = DatasetLibrary("datasets").download(DATASETS.BATTLEDIM)

Dataset already downloaded


We can now define the Benchmark and its parameters:

In [3]:
from ldimbenchmark import LDIMBenchmark

hyperparameters = {
    "NEWMETHOD": {
        "battledim": {
            "random": 0.1,
        }
    }
}

benchmark = LDIMBenchmark(
    hyperparameters,
    datasets,
    results_dir="./benchmark-results",
    debug=True,
)
benchmark.add_local_methods([MyNewLeakageDetectionMethod()])

Finally we can run and evaluate the benchmark:

In [4]:
# execute benchmark
benchmark.run_benchmark()

benchmark.evaluate()

                              TP    FP  TN      FN  TTD      wrongpipe    score    precision    recall (TPR)    TNR    FPR    FNR    F1
--------------------------  ----  ----  ----  ----  -----  -----------  -------  -----------  --------------  -----  -----  -----  ----
('battledim', 'NEWMETHOD')     0     0          19                   0        0          nan               0    nan    nan      1   nan
